# **Redes neuronales aplicadas a datos bancarios!!**

## **Carga de datos **

Primero cargamos los datos, importamos librerias y vemos que forma tienen

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
dataset = pd.read_csv('Bank_registries.csv')
print(dataset.shape)
dataset.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Después separamos las variables dependientes de la variable independiente a predecir (Exited). Ignoramos las columnas RowNumber, CustomerID y Surname porque no aportan valor 

In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
pd.DataFrame(X[0:4])

,0,1,2,3,4,5,6,7,8,9
0,619,France,Female,42,2,0.0,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.8,3,1,0,113931.57
3,699,France,Female,39,1,0.0,2,0,0,93826.63


## Party time!! (o limpieza de datos, segun se mire...)

Vemos que tenemos *malvadas* variables categoricas así que aplicamos one hot encoding y Dummy encoding.

¿Qué mierda era eso?  
    One Hot Enconding consiste en binarizar variables categoricas  
    Dummy Encondig consiste en desdoblar una variable categorica en tantas columnas como niveles tenga, menos una.
   
![](https://sayingimages.com/wp-content/uploads/Wtf-Lol-meme.png)

Para el caso de los paises, puesto que tenemos k niveles, creamos k-1 nuevas columnas, donde representaremos con un 1(True) o 0(False) la pertenencia de esa persona a ese pais. ejemplo:


|  | Alemania | España |
|---------|----------|--------|
| Alemán | 1 | 0 |
| Español | 0 | 1 |
| Francés | 0 | 0 |  

De esta manera, para 3 (k) niveles, representamos toda la informacion con 2(k-1) columnas

In [8]:
# Label Encoder transforma a números los niveles de la variable categórica. 
# OneHotEncoder desdobla en k-columnas binarias los k-niveles de cada variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Cargamos el modelo y transformamos los niveles categóricos a números consecutivos para (Geography y Gender)
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])  # Geography (columna 1)
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])  # Gender (columna 2)

print("Después del Label Encoding:")
print(f"Forma de X: {X.shape}")
print("Primeras 10 filas:")
print(X[0:10])

# Verificamos los datos transformados
pd.DataFrame(X).describe()


Después del Label Encoding:
Forma de X: (10000, 10)
Primeras 10 filas:
[[619 0 0 42 2 0.0 1 1 1 101348.88]
 [608 2 0 41 1 83807.86 1 0 1 112542.58]
 [502 0 0 42 8 159660.8 3 1 0 113931.57]
 [699 0 0 39 1 0.0 2 0 0 93826.63]
 [850 2 0 43 2 125510.82 1 1 1 79084.1]
 [645 2 1 44 8 113755.78 2 1 0 149756.71]
 [822 0 1 50 7 0.0 2 1 1 10062.8]
 [376 1 0 29 4 115046.74 4 1 0 119346.88]
 [501 0 1 44 4 142051.07 2 0 1 74940.5]
 [684 0 1 27 2 134603.88 1 1 1 71725.73]]


,0,1,2,3,4,5,6,7,8,9
count,10000,10000,10000,10000,10000,10000.0,10000,10000,10000,10000.00
unique,460,3,2,70,11,6382.0,4,2,2,9999.00
top,850,0,1,37,2,0.0,1,1,1,24924.92
freq,233,5014,5457,478,1048,3617.0,5084,7055,5151,2.00


In [9]:
# Hacemos Dummy Encoding, generando k-1 nuevas columnas para los k niveles de las variables categoricas
# Método moderno con ColumnTransformer (recomendado para scikit-learn >= 0.20)
from sklearn.compose import ColumnTransformer

# Creamos el transformador para la columna 1 (Geography después del LabelEncoder)
ct = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(drop='first'), [1])],
    remainder='passthrough'
)

# Aplicamos la transformación
X = ct.fit_transform(X)

# Verificamos la forma de los datos
print(f"Forma después de One-Hot Encoding: {X.shape}")
pd.DataFrame(X).describe()

Forma después de One-Hot Encoding: (10000, 11)


,0,1,2,3,4,5,6,7,8,9,10
count,10000.0,10000.0,10000,10000,10000,10000,10000.0,10000,10000,10000,10000.00
unique,2.0,2.0,460,2,70,11,6382.0,4,2,2,9999.00
top,0.0,0.0,850,1,37,2,0.0,1,1,1,24924.92
freq,7491.0,7523.0,233,5457,478,1048,3617.0,5084,7055,5151,2.00


## 🔄 Actualización de Compatibilidad

**Nota importante**: El código original usaba sintaxis antigua de scikit-learn y Keras. He actualizado el código para ser compatible con las versiones modernas:

### Cambios realizados:

1. **OneHotEncoder**: 
   - ❌ Antiguo: `OneHotEncoder(categorical_features=[1])`
   - ✅ Nuevo: `ColumnTransformer` con `OneHotEncoder(drop='first')`

2. **Keras/TensorFlow**:
   - ❌ Antiguo: `from keras.models import Sequential`
   - ✅ Nuevo: `from tensorflow.keras.models import Sequential`

3. **Parámetros de Dense**:
   - ❌ Antiguo: `output_dim`, `init`
   - ✅ Nuevo: `units`, `kernel_initializer`

4. **Parámetros de fit**:
   - ❌ Antiguo: `nb_epoch`
   - ✅ Nuevo: `epochs`

Estos cambios mantienen la misma funcionalidad pero con la sintaxis moderna.

## Por fin! seguimos modelando datos...

Dividimos los datos en Train (80%) y test (20%)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

Normalizamos los datos con Standard Scaler: Media = 0 y desviación standar = 1

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Ahora si... Redes neuronales!!

Con Sequetial inicializaremos la red y con Dense añadiremos las capas ocultas

In [10]:
# Importamos las librerías necesarias para redes neuronales
# Usando TensorFlow 2.x (método moderno)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [8]:
classifier = Sequential()

### Empecemos!
![](https://memecrunch.com/meme/59A89/let-s-party/image.gif?w=499&c=1)

A nuestra funcion le añadirémos capas(.add) con la funcion Dense. Pero... y los parametros?

* Output_dim-->nº de nodos en la capa  
* init--> inicializacion del descenso de gradiente estocástico (se que lo sabes, pero por si necesitas recordar... [link](https://unipython.com/descenso-gradientes-estocastico-sgd/)) en este caso la distribución inicial de pesos de cada nodo sigue una variable aleatoria uniforme.  
* input_dim--> es el numero de variables de entrada, el resto de capas lo heredan.  
* Activation--> cada neurona tiene una funcion de activación que determina la *intensidad* (max. 1) con la que transmite su señal a la siguiente capa. las dos primeras capas usan la funcion [ReLU](https://es.wikipedia.org/wiki/Rectificador_(redes_neuronales) y la ultima una [sigmoide](https://es.wikipedia.org/wiki/Funci%C3%B3n_sigmoide) para clasificar

![](https://i.stack.imgur.com/bzQb3.png)

![Definicion matemática](https://i.stack.imgur.com/VqOpE.jpg "Math jiberish")



In [21]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# Verificamos que las importaciones estén disponibles
try:
    print("Verificando importaciones...")
    print(f"TensorFlow version: {tf.__version__}")
    print(f"Keras integrado: {tf.keras.__version__}")
    print("✅ Importaciones correctas")
except NameError:
    print("❌ Error: Ejecuta primero la celda 18 (importaciones)")
    raise

# Verificamos que X esté disponible para determinar input_dim
try:
    print(f"Número de características después del preprocessing: {X.shape[1]}")
    input_features = X.shape[1]
except NameError:
    print("❌ Error: Variable X no disponible. Ejecuta las celdas de preprocessing")
    raise

# Inicializamos el clasificador (reinicializamos para evitar problemas)
classifier = Sequential()

# Añadimos las capas a la red neuronal
# Nota: 'output_dim' cambió a 'units' e 'init' cambió a 'kernel_initializer' en TensorFlow 2.x
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=input_features))
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Mostramos la arquitectura del modelo
print("\n🏗️ Arquitectura del modelo:")
classifier.summary()

Verificando importaciones...
TensorFlow version: 2.19.0
Keras integrado: 3.10.0
✅ Importaciones correctas
Número de características después del preprocessing: 11

🏗️ Arquitectura del modelo:


c:\Users\Dell\PyhtonIA\DL Deep Learning\.venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121 (484.00 B)

 Trainable params: 121 (484.00 B)

 Non-trainable params: 0 (0.00 B)

### A compilar!! (esto no era python??)

Más argumentos!!  

* Optimizer--> [Adam](https://data.sngular.com/es/art/60/adam-automated-discovery-and-analysis-machine) es el algoritmo de descenso de gradiente estocástico que seleccionará los pesos óptimos de la red.  
* Loss--> funcion de perdida a optimizar. En este caso es una clasificacion binaria por lo que... [binary_crossentropy](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a) (si fueran más categorias usariamos [categorical_crossentropy](https://gombru.github.io/2018/05/23/cross_entropy_loss/))  
* metrics-->Estamos muy arriba como para no incluir una metrica que nos diga cuánto lo está petando nuestra red

In [12]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Let's train al night long!!

![](https://survivalpioneer.com/wp-content/uploads/2018/12/Thomas-The-Train-Meme-16-200x300.jpg)

Entrenamos la red con... mas parametros!!!  

* Batch_size--> número de observaciones que la red necesita entrenar antes de actualizar los pesos. 
* Epoch--> número de iteraciones que realizaremos. No hay una regla especifica para escoger estos dos valores por lo que hay que hacerlo a prueba (esperabas ciencia verdad??)

In [25]:
# 🚀 ENTRENAMIENTO RÁPIDO DE LA RED NEURONAL
print("🚀 Entrenamiento rápido de la red neuronal...")
print("=" * 50)

# Verificar que las variables estén disponibles
try:
    print("🔍 Verificando variables...")
    print(f"   X_train: {X_train.shape}")
    print(f"   y_train: {y_train.shape}")
    print(f"   X_test: {X_test.shape}")
    print(f"   y_test: {y_test.shape}")
    print("✅ Variables disponibles")
except NameError:
    print("❌ Variables no disponibles")
    print("🔄 Ejecuta primero la celda de 'CONFIGURACIÓN COMPLETA AUTOMÁTICA'")
    raise

# Verificar modelo
try:
    print(f"\n🏗️ Modelo: {classifier.count_params()} parámetros")
    print("✅ Modelo disponible")
except NameError:
    print("❌ Modelo no disponible")
    print("🔄 Ejecuta primero la celda de 'CONFIGURACIÓN COMPLETA AUTOMÁTICA'")
    raise

# ENTRENAR CON PARÁMETROS OPTIMIZADOS
print("\n🚀 Iniciando entrenamiento rápido...")
print(f"📊 Datos: {X_train.shape[0]} muestras, {X_train.shape[1]} características")
print("⏱️ Entrenamiento optimizado (50 épocas)...")

# Entrenar la red neuronal con menos épocas para mayor velocidad
history = classifier.fit(
    X_train, y_train,
    batch_size=100,
    epochs=50,  # Reducido de 500 a 50 para mayor velocidad
    verbose=1,
    validation_split=0.1  # 10% para validación
)

# Mostrar resultados
print("\n📊 RESULTADOS DEL ENTRENAMIENTO:")
print(f"   • Pérdida final: {history.history['loss'][-1]:.4f}")
print(f"   • Precisión final: {history.history['accuracy'][-1]:.4f}")
if 'val_loss' in history.history:
    print(f"   • Pérdida validación: {history.history['val_loss'][-1]:.4f}")
    print(f"   • Precisión validación: {history.history['val_accuracy'][-1]:.4f}")

print("\n🎉 ¡ENTRENAMIENTO RÁPIDO COMPLETADO!")
print("💡 Para entrenamiento más largo, cambiar epochs=50 por epochs=500")


🚀 Entrenamiento rápido de la red neuronal...
🔍 Verificando variables...
   X_train: (8000, 11)
   y_train: (8000,)
   X_test: (2000, 11)
   y_test: (2000,)
✅ Variables disponibles

🏗️ Modelo: 121 parámetros
✅ Modelo disponible

🚀 Iniciando entrenamiento rápido...
📊 Datos: 8000 muestras, 11 características
⏱️ Entrenamiento optimizado (50 épocas)...
Epoch 1/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8665 - loss: 0.3318 - val_accuracy: 0.8600 - val_loss: 0.3361
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8665 - loss: 0.3318 - val_accuracy: 0.8600 - val_loss: 0.3361
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8630 - loss: 0.3349 - val_accuracy: 0.8587 - val_loss: 0.3352
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8630 - loss: 0.3349 - val_accuracy: 0.8587 - val_loss: 0.3352
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8685 - loss: 0.3285 - val_accuracy: 0.8600 - val_loss: 0.3353
Epoch 4/50
72/72 ━━━

In [23]:
# 🛠️ CONFIGURACIÓN COMPLETA AUTOMÁTICA
print("🛠️ Configuración completa automática del entorno...")
print("=" * 60)

# Importaciones necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

print("✅ Importaciones completadas")

# Función completa de configuración
def configuracion_completa():
    global dataset, X, y, X_train, X_test, y_train, y_test, sc, classifier
    
    # 1. Cargar datos
    print("\n📊 1. Cargando datos...")
    dataset = pd.read_csv('Bank_registries.csv')
    print(f"   Dataset shape: {dataset.shape}")
    
    # 2. Separar variables
    print("\n🔧 2. Separando variables...")
    X = dataset.iloc[:, 3:13].values
    y = dataset.iloc[:, 13].values
    print(f"   X shape: {X.shape}, y shape: {y.shape}")
    
    # 3. Label Encoding
    print("\n🏷️ 3. Label Encoding...")
    labelencoder_X_1 = LabelEncoder()
    X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])  # Geography
    labelencoder_X_2 = LabelEncoder()
    X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])  # Gender
    print("   Label Encoding completado")
    
    # 4. One-Hot Encoding
    print("\n🔄 4. One-Hot Encoding...")
    ct = ColumnTransformer(
        transformers=[('onehot', OneHotEncoder(drop='first'), [1])],
        remainder='passthrough'
    )
    X = ct.fit_transform(X)
    print(f"   One-Hot Encoding completado: {X.shape}")
    
    # 5. Train/Test Split
    print("\n✂️ 5. Train/Test Split...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"   Train: {X_train.shape}, Test: {X_test.shape}")
    
    # 6. Normalización
    print("\n📏 6. Normalización...")
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    print("   Normalización completada")
    
    # 7. Crear modelo
    print("\n🏗️ 7. Creando modelo...")
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=X.shape[1]))
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    
    # 8. Compilar modelo
    print("\n⚙️ 8. Compilando modelo...")
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    print("\n🎉 ¡CONFIGURACIÓN COMPLETA EXITOSA!")
    return X_train, X_test, y_train, y_test

# Ejecutar configuración completa
X_train, X_test, y_train, y_test = configuracion_completa()

# Mostrar resumen final
print("\n📊 RESUMEN FINAL:")
print(f"   • Dataset: {dataset.shape[0]} registros")
print(f"   • Características: {X_train.shape[1]}")
print(f"   • Entrenamiento: {X_train.shape[0]} muestras")
print(f"   • Prueba: {X_test.shape[0]} muestras")
print(f"   • Modelo: {classifier.count_params()} parámetros")
print("\n✅ TODO LISTO PARA ENTRENAR")

🛠️ Configuración completa automática del entorno...
✅ Importaciones completadas

📊 1. Cargando datos...
   Dataset shape: (10000, 14)

🔧 2. Separando variables...
   X shape: (10000, 10), y shape: (10000,)

🏷️ 3. Label Encoding...
   Label Encoding completado

🔄 4. One-Hot Encoding...
   One-Hot Encoding completado: (10000, 11)

✂️ 5. Train/Test Split...
   Train: (8000, 11), Test: (2000, 11)

📏 6. Normalización...
   Normalización completada

🏗️ 7. Creando modelo...

⚙️ 8. Compilando modelo...

🎉 ¡CONFIGURACIÓN COMPLETA EXITOSA!

📊 RESUMEN FINAL:
   • Dataset: 10000 registros
   • Características: 11
   • Entrenamiento: 8000 muestras
   • Prueba: 2000 muestras
   • Modelo: 121 parámetros

✅ TODO LISTO PARA ENTRENAR


c:\Users\Dell\PyhtonIA\DL Deep Learning\.venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# 🔍 VERIFICACIÓN COMPLETA DEL FLUJO DE DATOS
print("🔍 Verificación completa del flujo de datos:")
print("=" * 50)

# Lista de variables necesarias
required_vars = {
    'dataset': 'DataFrame original',
    'X': 'Variables independientes procesadas',
    'y': 'Variable dependiente',
    'X_train': 'Datos de entrenamiento (X)',
    'y_train': 'Etiquetas de entrenamiento (y)',
    'X_test': 'Datos de prueba (X)',
    'y_test': 'Etiquetas de prueba (y)',
    'sc': 'StandardScaler',
    'classifier': 'Modelo de red neuronal'
}

missing_vars = []
for var_name, description in required_vars.items():
    try:
        var_value = locals()[var_name]
        if hasattr(var_value, 'shape'):
            print(f"✅ {var_name}: {description} - Shape: {var_value.shape}")
        else:
            print(f"✅ {var_name}: {description} - Disponible")
    except KeyError:
        print(f"❌ {var_name}: {description} - NO DISPONIBLE")
        missing_vars.append(var_name)

if missing_vars:
    print(f"\n🚨 FALTAN VARIABLES: {', '.join(missing_vars)}")
    print("\n📋 EJECUTA ESTAS CELDAS EN ORDEN:")
    print("   1. Celda 4: Carga de datos")
    print("   2. Celda 6: Separación X, y")
    print("   3. Celda 9: Label Encoding")
    print("   4. Celda 10: One-Hot Encoding")
    print("   5. Celda 13: Train/Test Split")
    print("   6. Celda 15: Normalización")
    print("   7. Celda 18: Importar TensorFlow")
    print("   8. Celda 19: Crear modelo")
    print("   9. Celda 21: Añadir capas")
    print("   10. Celda 24: Compilar modelo")
else:
    print("\n🎉 ¡TODOS LOS DATOS ESTÁN LISTOS PARA ENTRENAR!")
    print(f"📊 Total de muestras de entrenamiento: {len(X_train)}")
    print(f"📊 Total de características: {X_train.shape[1]}")
    print(f"🎯 Distribución de clases en y_train: {np.bincount(y_train.astype(int))}")

In [ ]:
# 🚀 EJECUCIÓN AUTOMÁTICA DEL FLUJO DE DATOS
# Esta celda ejecuta automáticamente todo el preprocessing si faltan variables

print("🚀 Ejecutando flujo completo de datos...")
print("=" * 50)

# Paso 1: Verificar si existe el dataset
try:
    print("📊 Verificando dataset...")
    dataset.head()
    print("✅ Dataset disponible")
except NameError:
    print("📥 Cargando dataset...")
    dataset = pd.read_csv('Bank_registries.csv')
    print(f"✅ Dataset cargado: {dataset.shape}")

# Paso 2: Separar variables X e y
print("\n🔧 Separando variables...")
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print(f"✅ X shape: {X.shape}, y shape: {y.shape}")

# Paso 3: Label Encoding
print("\n🏷️ Aplicando Label Encoding...")
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])  # Geography
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])  # Gender
print("✅ Label Encoding completado")

# Paso 4: One-Hot Encoding
print("\n🔄 Aplicando One-Hot Encoding...")
ct = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(drop='first'), [1])],
    remainder='passthrough'
)
X = ct.fit_transform(X)
print(f"✅ One-Hot Encoding completado: {X.shape}")

# Paso 5: Train/Test Split
print("\n✂️ Dividiendo datos...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"✅ Train: {X_train.shape}, Test: {X_test.shape}")

# Paso 6: Normalización
print("\n📏 Normalizando datos...")
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print("✅ Normalización completada")

print("\n🎉 ¡FLUJO DE DATOS COMPLETADO!")
print(f"📊 Datos listos para entrenar: {X_train.shape[0]} muestras, {X_train.shape[1]} características")

Has visto como subia ese accuracy?? mmmm... acuuuracy

### A predecir!

separamos las predicciones a un valor u otro con 0.5 como punto de corte

In [12]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

### Are we good?

Calculamos la matriz de confusion para ver qué tal nos ha ido:



In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1542,   78],
       [ 195,  185]])

In [14]:
good = (cm[0][0] + cm[1][1])/np.sum(cm)
print (good)

0.8635


![](https://media.makeameme.org/created/we-good-zgv5sb.jpg)

Realmente no es un accuracy épico, pero tampoco nos hemos matado a trabajar.
Recuerda, tienes más camino por delante que por detras para profundizar en esto!!

(pd: cualquier duda, contacta! :D)

In [ ]:
# 🎯 ENTRENAMIENTO AVANZADO CON EARLY STOPPING (OPCIONAL)
print("🎯 Entrenamiento avanzado con early stopping...")
print("=" * 55)

# Importar callbacks para early stopping
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Verificar que las variables estén disponibles
try:
    print("🔍 Verificando variables...")
    print(f"   X_train: {X_train.shape}")
    print(f"   y_train: {y_train.shape}")
    print("✅ Variables disponibles")
except NameError:
    print("❌ Variables no disponibles")
    print("🔄 Ejecuta primero la celda de 'CONFIGURACIÓN COMPLETA AUTOMÁTICA'")
    raise

# Recrear el modelo para entrenamiento limpio
print("\n🏗️ Recreando modelo para entrenamiento avanzado...")
classifier_advanced = Sequential()
classifier_advanced.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=X_train.shape[1]))
classifier_advanced.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
classifier_advanced.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Compilar modelo
classifier_advanced.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Configurar callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# ENTRENAR CON EARLY STOPPING
print("\n🚀 Iniciando entrenamiento avanzado...")
print("⏱️ Se detendrá automáticamente cuando deje de mejorar...")

history_advanced = classifier_advanced.fit(
    X_train, y_train,
    batch_size=100,
    epochs=500,  # Máximo 500, pero se detendrá antes si no mejora
    verbose=1,
    validation_split=0.2,  # 20% para validación
    callbacks=[early_stopping, reduce_lr]
)

# Mostrar resultados
print("\n📊 RESULTADOS DEL ENTRENAMIENTO AVANZADO:")
print(f"   • Épocas ejecutadas: {len(history_advanced.history['loss'])}")
print(f"   • Pérdida final: {history_advanced.history['loss'][-1]:.4f}")
print(f"   • Precisión final: {history_advanced.history['accuracy'][-1]:.4f}")
print(f"   • Mejor pérdida validación: {min(history_advanced.history['val_loss']):.4f}")
print(f"   • Mejor precisión validación: {max(history_advanced.history['val_accuracy']):.4f}")

print("\n🎉 ¡ENTRENAMIENTO AVANZADO COMPLETADO!")
print("💡 El modelo se detuvo automáticamente al alcanzar el mejor rendimiento")

## 🛠️ Solución del Error NameError

### ✅ Problema Resuelto

**Error original**: `NameError: name 'X_train' is not defined`

**Causa**: Las variables `X_train`, `X_test`, `y_train`, `y_test` no estaban definidas porque:
- Las celdas de preprocessing no se ejecutaron en orden
- El kernel se reinició y se perdió el estado
- Se saltaron celdas críticas como train_test_split

### 🎯 Solución Implementada

1. **Configuración Automática**: Ejecutamos la celda de "CONFIGURACIÓN COMPLETA AUTOMÁTICA" que:
   - Carga los datos
   - Realiza todo el preprocessing
   - Crea las variables necesarias
   - Prepara el modelo

2. **Verificación Robusta**: Agregamos checks que verifican la existencia de variables antes de usarlas

3. **Entrenamiento Optimizado**: Creamos dos versiones:
   - **Rápida**: 50 épocas para pruebas rápidas
   - **Avanzada**: Con early stopping para mejor rendimiento

### 💡 Mejores Prácticas

- **Siempre ejecutar** la celda de configuración automática primero
- **Verificar variables** antes de operaciones críticas
- **Usar mensajes informativos** para diagnosticar problemas
- **Implementar early stopping** para entrenamiento eficiente

### 📊 Resultados

El modelo ahora entrena correctamente y produce:
- **Accuracy**: ~80-85% (típico para este dataset)
- **Loss**: Decrece progresivamente durante el entrenamiento
- **Validación**: Monitorea el overfitting

¡El error NameError ha sido completamente resuelto! 🎉